<a href="https://colab.research.google.com/github/tryfasting/deeplearning-pytorch-textbook/blob/main/08-03%20%EA%B9%8A%EC%9D%80%20CNN%EC%9C%BC%EB%A1%9C%20MNIST%20%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 앞서 배운 CNN에 층을 더 추가하여 MNIST를 분류해보자.

### 1. 모델 이해하기

우리가 만들 모델의 아키텍처를 이해해보자.  
모델의 아키텍처는 총 5개의 층으로 구성된다.  
앞서 배운 챕터에서 1번, 2번 레이어는 동일하되, 새로운 합성곱층과 전결합층을 추가한다.  모델의 아키텍처는 총 5개의 층으로 구성할 예정이다.

### * 왜 Fully-Connected Layer가 CNN의 모델에 추가될까?

Fully-Connected Layer는 분류 부분에 해당한다.
- CNN의 convolution layer와 pooling layer는 local features를 추출하는 역할을 한다.Fully-Connected Layer는 추출된 다양한 특징들을 종합적으로 해석하고 통합하는 역할을 한다.

- spatial information -> semantic information으로 변환.
예를 들어, '이 부분에 눈이 있고, 저 부분에 코가 있다.'는 공간적 정보를 '이것은 얼굴이다.'라는 의미적 판단으로 연결시킨다.

- 차원 축소와 최종 출력 생성
Fully-Connected Layer는 고차원 데이터를 flatten한 뒤, 단계적으로 차원을 축소시켜 최종 출력(예: 10개의 숫자 클래스)로 매핑하는 역할을 한다.

### * CNN과 fully-connected layer의 역할 구분은 의도적 설계인가? 아니면 경험적 발견인가?

*역사적 발전 과정  
CNN의 기본 개념은 처음부터 생물학적 영감에서 출발했습니다. 1980년대 Kunihiko Fukushima와 Yann LeCun의 초기 연구는 인간 시각 시스템의 계층적 특성에서 영감을 받았습니다. LeCun은 1989년 "Backpropagation Applied to Handwritten Zip Code Recognition"에서 이 개념을 구체화했고, 1990년대에 LeNet-5 아키텍처로 발전시켰습니다.

*의도적 설계 요소:

- 공간적 구조 인식: CNN의 설계는 처음부터 이미지와 같은 고차원 입력의 공간적 구조를 활용하기 위한 의도적 결정이었습니다. "고차원 입력의 경우, 이전 볼륨의 모든 뉴런에 모든 뉴런을 연결하는 것은 데이터의 공간적 구조를 고려하지 않기 때문에 비실용적입니다"라는 인식이 있었습니다.

- 파라미터 효율성: CNN은 파라미터 공유를 통해 더 적은 수의 가중치로 학습할 수 있도록 의도적으로 설계되었습니다. 이는 "고차원 입력 데이터에 도움이 됩니다".

- 지역적 특징 추출: 컨볼루션 레이어가 지역적 특징을 추출하고 fully-connected 레이어가 전역적 패턴을 인식하는 상호보완적 역할은 어느 정도 의도적으로 설계되었습니다.

*경험적 발견과 사후적 이해:

- 아키텍처 패턴 발전: "INPUT => CONV => RELU => FC => SOFTMAX"와 같은 일반적인 CNN 아키텍처 패턴은 시행착오와 경험적 연구를 통해 점진적으로 발전했습니다.

- 층 간 상호작용 이해: Fully-connected 레이어가 "추출된 특징을 바탕으로 고수준 추론과 의사결정을 수행"한다는 이해는 모델 사용 경험에서 얻은 통찰입니다.

- 다양한 변형 아키텍처: MNIST, CIFAR-10 같은 데이터셋과 ImageNet 대회를 통해 다양한 CNN 변형 아키텍처가 등장했는데, 이는 실험과 경쟁을 통한 경험적 발견의 산물입니다.

*균형 잡힌 관점
CNN 아키텍처의 발전은 의도적 설계와 경험적 발견의 상호작용 결과입니다. 초기 설계는 생물학적 영감과 이론적 원칙에 기반했지만, 구체적인 아키텍처 패턴과 각 레이어의 최적 배치는 실험과 경쟁을 통해 점진적으로 발전했습니다.

예를 들어, fully-connected 레이어가 CNN의 마지막에 위치하는 것은 "CNN의 컨볼루션 레이어와 풀링 레이어가 추출한 특징을 종합하여 분류 작업을 수행"하는 논리적 설계이면서도, 다양한 구성을 시험한 결과 이 방식이 가장 효과적이라는 경험적 증거가 뒷받침됩니다.

결론적으로, CNN과 fully-connected 레이어의 역할 구분은 이론적 원칙에 기반한 의도적 설계와 실제 적용을 통한 경험적 발견이 함께 진화한 결과라고 볼 수 있습니다.

### 2. 깊은 CNN으로 MNIST 분류하기

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [2]:
device = 'cpu' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

하이퍼파라미터를 설정한다.

In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

데이터셋을 정의한다.  

In [4]:
mnist_train = dsets.MNIST(root='MNIST_data/', #다운로드 경로 지정
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

100%|██████████| 9.91M/9.91M [00:01<00:00, 5.11MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 135kB/s]
100%|██████████| 1.65M/1.65M [00:06<00:00, 247kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.48MB/s]


데이터로더를 사용하여 배치 크기를 지정해준다.

In [5]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

클래스로 모델을 설계한다.

In [6]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5        # 드롭아웃 확률
        # 입력 이미지 형태 : (?, 28, 28, 1)


        # L1 : 첫 번째 합성곱층(Conv Layer)
        # Conv2d : 출력 채널 32개, 커널 크기 3*3, 스트라이드 1, 패딩 1
        # ReLU : 활성화 함수
        # MaxPool2d: 커널 크기 2*2, 스트라이드 2로 다운샘플링
        # -> 출력형태 : (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # 입력 이미지 형태 : (?, 14, 14, 32)
        # L2 : 두 번째 합성곱층 (Conv Layer)
        # Conv2d: 출력 채널 64개, 커널 크기 3x3, 스트라이드 1, 패딩 1
        # ReLU: 활성화 함수
        # MaxPool2d: 커널 크기 2x2, 스트라이드 2로 다운샘플링
        # -> 출력형태 : (?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # 입력 이미지 형태 : (?, 7, 7, 64)
        # Conv2d: 출력 채널 128개, 커널 크기 3x3, 스트라이드 1, 패딩 1
        # ReLU: 활성화 함수
        # MaxPool2d: 커널 크기 2x2, 스트라이드 2, 패딩 1로 다운샘플링
        # -> 출력 형태: (?, 4, 4, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        )

        # 입력 노드 수 : 4*4*128, 출력 노드 수 : 625
        # L4 : 첫 번째 Fully connected layer
        # ReLU : 활성화 함수
        # Dropout : 드롭아웃으로 과적합 방지, p = 0.5
        self.fc1 = torch.nn.Linear(4*4*128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight) # 가중치 초기화

        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p = 1 - self.keep_prob)
        )

        # 입력 노드 수 : 625, 출력 노드 수 : 10, (클래스 개수)
        # L5 : 최종 선형층, fully-connected layer
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight) # 가중치 초기화

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1) # 선형층 입력 위해 flatten
        out = self.layer4(out)
        out = self.fc2(out)
        return out

모델을 정의한다.

In [7]:
# CNN 모델 정의
model = CNN().to(device)

비용 함수와 옵티마이저를 정의한다.

In [8]:
criterion = torch.nn.CrossEntropyLoss().to(device) # 비용함수에 소프트맥스 함수 포함
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

총 배치의 수를 출력해보겠습니다.

In [9]:
total_batch = len(data_loader)
print(f'총 배치의 수 : {total_batch}')

총 배치의 수 : 600


In [10]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X,Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니배치, Y는 레이블
        # image is already size of (28*28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothetis = model(X)
        cost = criterion(hypothetis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost/ total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.19235523
[Epoch:    2] cost = 0.0550963804
[Epoch:    3] cost = 0.0382400788
[Epoch:    4] cost = 0.0306283627
[Epoch:    5] cost = 0.0248726383
[Epoch:    6] cost = 0.0206133127
[Epoch:    7] cost = 0.0169408526
[Epoch:    8] cost = 0.0154811479
[Epoch:    9] cost = 0.0149189457
[Epoch:   10] cost = 0.0119884657
[Epoch:   11] cost = 0.00975780655
[Epoch:   12] cost = 0.00903648324
[Epoch:   13] cost = 0.00889740698
[Epoch:   14] cost = 0.00743382238
[Epoch:   15] cost = 0.00752810063


테스트를 해보자.

In [11]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()

with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, dim=1) ==  Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy : ', accuracy.item())

/usr/local/lib/python3.11/dist-packages/torchvision/datasets/mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.11/dist-packages/torchvision/datasets/mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy :  0.9868000149726868


층을 쌓는다고 늘 정확도가 더 높아지지는 않는다.  
모델 구조와 데이터셋의 특성, 그리고 환경 차이에 따라 달라질 수도 있다.